In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
# cd
import os
path = os.getcwd()
print(path)

/Users/kanazawaatsuya/imp/MQ2007/RankNet


In [2]:
from ptranking.data.data_utils import LTRDataset, SPLIT_TYPE
data_id = 'MQ2007_Super'
#MQ2007内にあるもの
fold1 = LTRDataset(SPLIT_TYPE.Train, data_id=data_id, file='S1.txt', batch_size=1, shuffle=True, presort=True, data_dict=None, eval_dict=None, buffer=False)
fold2 = LTRDataset(SPLIT_TYPE.Train, data_id=data_id, file='S2.txt', batch_size=1, shuffle=True, presort=True, data_dict=None, eval_dict=None, buffer=False)
fold3 = LTRDataset(SPLIT_TYPE.Train, data_id=data_id, file='S3.txt', batch_size=1, shuffle=True, presort=True, data_dict=None, eval_dict=None, buffer=False)
fold4 = LTRDataset(SPLIT_TYPE.Train, data_id=data_id, file='S4.txt', batch_size=1, shuffle=True, presort=True, data_dict=None, eval_dict=None, buffer=False)
fold5 = LTRDataset(SPLIT_TYPE.Train, data_id=data_id, file='S5.txt', batch_size=1, shuffle=True, presort=True, data_dict=None, eval_dict=None, buffer=False)

In [12]:
layers = [nn.Linear(46, 128),
          nn.ReLU(),
          nn.Linear(128, 64),
          nn.ReLU(),
          nn.Linear(64, 32),
          nn.ReLU(),
          nn.Linear(32, 1)]

In [13]:
class RankNet(nn.Module):
    def __init__(self, layers):
        super(RankNet, self).__init__()
        self.model = nn.Sequential(*layers)
        
    def forward(self, batch_ranking, label):
        batch_pred = self.model(batch_ranking) # [1, 40, 1]
        batch_sij = torch.squeeze(batch_pred, 0) - torch.squeeze(batch_pred, 2) # [40, 40]
        batch_pij = 1.0 / (1.0 + torch.exp(-batch_sij)) # [40, 40]

        label_dim = torch.squeeze(label)
        label_diffs = torch.unsqueeze(label_dim, 1) - label # [40,40]
        batch_Sij = torch.clamp(label_diffs, -1, 1)
        l_pij = 0.5 * (1.0 + batch_Sij)

        batch_loss = F.binary_cross_entropy(input=torch.triu(batch_pij, diagonal=1), target=torch.triu(l_pij, diagonal=1), reduction='mean')    
        
        return batch_loss
    
    def predict(self, x):
        return self.model(x)

In [14]:
# ndcg
def DCG(sorted_labels, cutoff):
    denoms = torch.log2(torch.arange(2, cutoff+2))
    nums = torch.pow(2, sorted_labels[0:cutoff])-1
    dcg = sum(nums / denoms)
    return dcg

def nDCG(ideal, pred, k):
    dcg_f = DCG(pred, k)
    dcg = DCG(ideal, k)
    nDCG = dcg_f / dcg
    return nDCG

def compute_ndcg(score, label, cutoff):
    batch_pred = model.predict(score)
    idx = torch.argsort(torch.squeeze(batch_pred),descending = True)
    pred_sorted_labels = torch.squeeze(label)[idx]
    ndcg = torch.nan_to_num(nDCG(torch.squeeze(label), pred_sorted_labels, cutoff))
    
    return ndcg

In [15]:
def testing(data):
    with torch.no_grad():
        ndcg_sum = 0
        for _,batch_ranking,label in data:
        #computes ndcg
            ndcg = compute_ndcg(batch_ranking,label,5)
            ndcg_sum += ndcg  
    ndcg_sum /= len(data)
    return(ndcg_sum)

In [16]:
def training_loop2(n_epochs,optimizer,model,train):
    
    
    # モデルの訓練
    for epoch in range(1, n_epochs + 1):
        
        # クエリごとの訓練
        batch_loss = 0
        batch_ndcg = 0
        for _,batch_ranking,label in train:
            batch_loss += model.forward(batch_ranking=batch_ranking, label=label)
            #print(compute_ndcg(batch_ranking,label,5))
            batch_ndcg += compute_ndcg(batch_ranking,label,5)
                    
        fold_loss_mean = batch_loss / len(fold1)
        fold_ndcg_mean = batch_ndcg / len(fold1)
        
        print('epoch:{0}, loss:{1}, ndcg:{2}'.format(epoch, fold_loss_mean, fold_ndcg_mean))
        
        optimizer.zero_grad()
        fold_loss_mean.backward()
        optimizer.step()
            

In [10]:
a,b,c=fold1[0]

In [21]:
print(b)
print(c)

tensor([[[0.0025, 0.3333, 0.0000,  ..., 0.5000, 0.5057, 0.0000],
         [0.0305, 0.1667, 0.0000,  ..., 0.3333, 0.2759, 0.0000],
         [0.0052, 0.0000, 0.2500,  ..., 0.6667, 0.3678, 0.0000],
         ...,
         [0.0163, 0.0000, 0.0000,  ..., 1.0000, 0.7471, 0.0000],
         [0.0291, 0.1667, 0.0000,  ..., 0.5000, 0.3793, 0.0000],
         [1.0000, 0.0000, 0.0000,  ..., 1.0000, 0.6897, 0.0000]]])
tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.]])


In [18]:
compute_ndcg(b,c,5)

tensor(0.5531)

In [9]:
fold1[0]

('10',
 tensor([[[0.0025, 0.3333, 0.0000,  ..., 0.5000, 0.5057, 0.0000],
          [0.0305, 0.1667, 0.0000,  ..., 0.3333, 0.2759, 0.0000],
          [0.0052, 0.0000, 0.2500,  ..., 0.6667, 0.3678, 0.0000],
          ...,
          [0.0163, 0.0000, 0.0000,  ..., 1.0000, 0.7471, 0.0000],
          [0.0291, 0.1667, 0.0000,  ..., 0.5000, 0.3793, 0.0000],
          [1.0000, 0.0000, 0.0000,  ..., 1.0000, 0.6897, 0.0000]]]),
 tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0.]]))

In [25]:
training_loop2(
    n_epochs = 100,
    optimizer = torch.optim.SGD(model.parameters(),lr = 0.01),
    model = model,
    train = fold1,
    )

epoch:1, loss:0.33804160356521606, ndcg:0.27715516090393066
epoch:2, loss:0.33804139494895935, ndcg:0.2775244116783142
epoch:3, loss:0.33804115653038025, ndcg:0.2775244116783142
epoch:4, loss:0.3380410075187683, ndcg:0.27750131487846375
epoch:5, loss:0.3380407691001892, ndcg:0.2770342230796814
epoch:6, loss:0.3380405604839325, ndcg:0.2770342230796814
epoch:7, loss:0.33804047107696533, ndcg:0.27706366777420044
epoch:8, loss:0.3380402624607086, ndcg:0.276743620634079
epoch:9, loss:0.3380400538444519, ndcg:0.27651485800743103
epoch:10, loss:0.3380397856235504, ndcg:0.27651485800743103
epoch:11, loss:0.33803969621658325, ndcg:0.27690187096595764
epoch:12, loss:0.3380395174026489, ndcg:0.2768726348876953
epoch:13, loss:0.3380393087863922, ndcg:0.2768190801143646
epoch:14, loss:0.3380391299724579, ndcg:0.2769017517566681
epoch:15, loss:0.33803895115852356, ndcg:0.2769017517566681
epoch:16, loss:0.3380386531352997, ndcg:0.277054101228714
epoch:17, loss:0.33803850412368774, ndcg:0.277054101228

In [7]:
def training_loop(n_epochs,optimizer,model,data_list):
    count = 0
    best_ndcg = 0
    
    while count < len(data_list):
        # 最初の3つをtrainに
        train = data_list[0]+data_list[1]+data_list[2]
        # 1つをvalidationに
        val = data_list[3]
        # 最後をtestに
        test = data_list[4]
        
        # モデルの訓練
        for epoch in range(1, n_epochs + 1):
        
            # クエリごとの訓練
            batch_loss = 0
            batch_ndcg = 0
            for _,batch_ranking,label in train:
                batch_loss += model.forward(batch_ranking=batch_ranking, label=label)
                #print(compute_ndcg(batch_ranking,label,5))
                batch_ndcg += compute_ndcg(batch_ranking,label,5)
                    
            fold_loss_mean = batch_loss / (len(data_list[0])+len(data_list[1])+len(data_list[2]))
            fold_ndcg_mean = batch_ndcg / (len(data_list[0])+len(data_list[1])+len(data_list[2]))
            
            if epoch ==1 or epoch%10==0:
                print('epoch:{0}, loss:{1}, ndcg:{2}'.format(epoch, fold_loss_mean, fold_ndcg_mean))
            
            # パラメータ選択
            val_loss = 0
            val_ndcg = 0
            for _,batch_ranking,label in val:
                val_loss += model.forward(batch_ranking=batch_ranking, label=label)
                val_ndcg += compute_ndcg(batch_ranking,label,5)
                
            val_loss_mean = val_loss / len(val)
            val_ndcg_mean = val_ndcg / len(val)    
                
            if val_ndcg_mean > best_ndcg:
                best_ndcg = val_ndcg_mean
            
                # 訓練したモデルの保存
                torch.save(model.state_dict(), 'weight.pth')
            
            optimizer.zero_grad()
            fold_loss_mean.backward()
            optimizer.step()
            
        # パラメータの読み込み
        param = torch.load('weight.pth')
        model.load_state_dict(param)
        
        #test
        test_ndcg = testing(test)
        
        print('test:{0}'.format(test_ndcg))
        
        #　右にひとつシフト ([1,2,3,4,5] -> [5,1,2,3,4]となる)
        data_list.insert(0,data_list.pop())
        
#         if epoch%10==0:
#             print("epoch: ", epoch, "loss: ", epoch_loss, "ndcg: ", epoch_ndcg)
        count += 1

In [8]:
data_list = [fold1,fold2,fold3,fold4,fold5]

In [17]:
# クラスのインスタンス化
model = RankNet(layers)

In [10]:
training_loop(
    n_epochs = 100,
    optimizer = torch.optim.SGD(model.parameters(),lr = 0.01),
    model = model,
    data_list = data_list,
    )

epoch:1, loss:0.33804816007614136, ndcg:0.2129535973072052
epoch:10, loss:0.3380464017391205, ndcg:0.21400970220565796
epoch:20, loss:0.33804401755332947, ndcg:0.21507927775382996
epoch:30, loss:0.3380418121814728, ndcg:0.21627427637577057
epoch:40, loss:0.33803996443748474, ndcg:0.218105748295784
epoch:50, loss:0.33803796768188477, ndcg:0.21888558566570282
epoch:60, loss:0.33803579211235046, ndcg:0.2204192578792572
epoch:70, loss:0.3380337357521057, ndcg:0.22214770317077637
epoch:80, loss:0.3380317687988281, ndcg:0.2235986590385437
epoch:90, loss:0.338029682636261, ndcg:0.224581778049469
epoch:100, loss:0.33802759647369385, ndcg:0.225375235080719
test:0.2294071763753891
epoch:1, loss:0.33805733919143677, ndcg:0.23191629350185394
epoch:10, loss:0.3380555808544159, ndcg:0.2341095507144928
epoch:20, loss:0.3380531668663025, ndcg:0.2361796349287033
epoch:30, loss:0.33805134892463684, ndcg:0.23738986253738403
epoch:40, loss:0.33804914355278015, ndcg:0.23919181525707245
epoch:50, loss:0.338